In [ ]:
import numpy as np
from pandas import read_csv
from javelin.structure import Structure

# path to the folder containing .csv input files generated by Ising.out
input_path = "sim"
# size o
supercell_size = 10
doping = 0.2
n_samples = 10
samples = []



"""Reading configurations created by Ising.out"""

def read_structure(size, doping, path):
    numbers = np.array([1, 2, 3, 4, 5]*size**3)
    # cubic perovskite basis
    pos = [(0, 0, 0), (0.5, 0, 0), (0, 0.5, 0), (0, 0, 0.5), (0.5, 0.5, 0.5)]
    # empirical doping dependence of the lattice constant
    cell = 4.35 - 0.17*doping
    # reading the structure
    structure = Structure(numbers=numbers, positions=pos*size**3, unitcell=cell)
    structure.atoms = read_csv(path)
    structure.reindex([size, size, size, 5])
    return structure


for i in range(n_samples):
    samples.append(read_structure(supercell_size, doping, input_path+"_"+str(i)+".csv"))
samples[0].atoms


In [ ]:

import xarray as xr

"""
Plotting a 2D slice of the Bi3+-Bi5+ sublattice. 
"""


def slice(structure, size,  k_value, site):
    img = xr.DataArray.from_series(structure.atoms.Z)
    img = img[dict(k=k_value, site=site)]
    num = []
    for i in range(size):
        for j in range(size):
            if int(img[i, j]) == 78:
                num.append("Cu")
            else:
                num.append("Au")
    slice = Structure(symbols=num, positions=[(0, 0, 0)]*size**2, unitcell=1)
    slice.reindex([size, size, 1, 1])
    atoms = slice.to_ase()
    return atoms

# plotting the slice
from ase.visualize.plot import plot_atoms
import matplotlib.pyplot as plt

plt.rcParams['axes.linewidth'] = 1
plt.rcParams["font.family"] = "Arimo"

# using ase to visualize atoms
atoms = slice(samples[0], 10, 4, 0)
fig, ax = plt.subplots(figsize=[8, 6])
ax.tick_params(labelsize=13)
ax.axis("off")
plot_atoms(atoms, ax, radii=0.3)

In [ ]:
# quantifying short range correlations

from javelin.neighborlist import NeighborList
from pandas import MultiIndex

"""
Calculating the chemical correlation coefficient for a given atom type and a pair of sites.
"""


def get_correlation(structure, site1, site2, atom, distance):
    range = 0.01
    count = 0
    while structure.get_neighbors(site1, site2, minD=distance-range, maxD=distance+range).values.size == 0:
        count += 1
        range *= count
    vectors = structure.get_neighbors(
        site1, site2, minD=distance-range, maxD=distance+range)
    vectors = np.asarray(vectors)
    count = 0
    total = 0
    match_count = 0
    for site1, site2, i, j, k in vectors:
        Z1 = structure.atoms.xs(site1, level='site').Z
        Z2 = structure.atoms.xs(site2, level='site').Z
        Z2 = Z2.reindex(MultiIndex.from_product(
            [np.roll(Z2.index.get_level_values(0).drop_duplicates(), i),
             np.roll(Z2.index.get_level_values(1).drop_duplicates(), j),
             np.roll(Z2.index.get_level_values(2).drop_duplicates(), k)],
            names=['i', 'j', 'k']))
        count += Z1.value_counts()[atom]
        total += Z1.size
        match_count += np.logical_and(Z1.values ==
                                      atom, Z2.values == atom).sum()
    theta = count/total
    return (match_count/total - theta*theta)/(theta*(1-theta))


In [ ]:
""" Calculating the distance dependence of correlation coefficient"""
distances = [1, 2, 3, 4, 5, 6, 7, 8, 9]
correlation = []
for d in distances:
    corr = 0
    for s in samples:
        corr += get_correlation(s, 0, 0, 78, d)
    corr = abs(corr/len(samples))
    correlation.append(corr)

    
plt.figure(figsize = [8,6])
plt.plot(distances,correlation, "--o", color = "red")
plt.xlabel("Distance [l.u.]", fontsize = 13)
plt.ylabel("Correlation", fontsize  = 14)

